# Knet RNN example
**TODO**: Use the new RNN interface, add dropout?

In [1]:
using Knet
True=true # so we can read the python params
include("common/params_lstm.py")

true

In [2]:
println("OS: ", Sys.KERNEL)
println("Julia: ", VERSION)
println("Knet: ", Pkg.dependencies()[Base.UUID("1902f260-5fb4-5aff-8c31-6271790ab950")].version)
println("GPU: ", read(`nvidia-smi --query-gpu=name --format=csv,noheader`,String))

OS: Linux
Julia: 1.5.0
Knet: 1.4.0
GPU: Quadro M2000
Tesla P4



In [3]:
# define model: this uses an outdated interface, see tutorial/70.imdb.ipynb for a more up-to-date implementation
function initmodel()
    rnn = RNN(EMBEDSIZE,NUMHIDDEN; rnnType=:gru, atype=Knet.array_type[])
    inputMatrix = param(EMBEDSIZE,MAXFEATURES)
    outputMatrix = param(2,NUMHIDDEN)
    return (rnn,inputMatrix,outputMatrix)
end;

In [4]:
# define loss and its gradient
function predict(model, inputs)
    rnn, inputMatrix, outputMatrix = model # (1,1,W), (X,V), (2,H)
    indices = permutedims(hcat(inputs...)) # (B,T)
    rnnInput = inputMatrix[:,indices] # (X,B,T)
    rnnOutput = rnn(rnnInput) # (H,B,T)
    return outputMatrix * rnnOutput[:,:,end] # (2,H) * (H,B) = (2,B)
end

loss(model,x,y)=nll(predict(model,x),y)

loss (generic function with 1 method)

In [5]:
# load data
include(Knet.dir("data","imdb.jl"))
@time (xtrn,ytrn,xtst,ytst,imdbdict)=imdb(maxlen=MAXLEN,maxval=MAXFEATURES)
for d in (xtrn,ytrn,xtst,ytst); println(summary(d)); end

┌ Info: Loading IMDB...
└ @ Main /userfiles/dyuret/.julia/dev/Knet/data/imdb.jl:57


  9.476542 seconds (26.67 M allocations: 1.390 GiB, 8.57% gc time)
25000-element Array{Array{Int32,1},1}
25000-element Array{Int8,1}
25000-element Array{Array{Int32,1},1}
25000-element Array{Int8,1}


In [6]:
imdbarray = Array{String}(undef,88584)
for (k,v) in imdbdict; imdbarray[v]=k; end
imdbarray[xtrn[1]]

150-element Array{String,1}:
 "reilly"
 "reilly"
 "reilly"
 "reilly"
 "reilly"
 "reilly"
 "reilly"
 "reilly"
 "reilly"
 "reilly"
 "reilly"
 "reilly"
 "reilly"
 ⋮
 "tommy"
 "davidson"
 "and"
 "damon"
 "wayans"
 "moving"
 "performances"
 "in"
 "spike"
 "lee's"
 "satire"
 "bamboozled"

In [7]:
# prepare for training
model = nothing; GC.gc(true); # Reclaim memory from previous run
model = initmodel()
dtrn = minibatch(xtrn,ytrn,BATCHSIZE;shuffle=true)

390-element Knet.Train20.Data{Tuple{Array{Array{Int32,1},1},Array{Int8,1}}}

In [8]:
# cold start
@time adam!((x,y)->loss(model,x,y), dtrn, lr=LR, beta1=BETA_1, beta2=BETA_2, eps=EPS)

 20.553846 seconds (27.02 M allocations: 1.438 GiB, 2.18% gc time)


In [9]:
# prepare for training
model = nothing; GC.gc(true); # Reclaim memory from previous run
model = initmodel()

(GRU(input=125,hidden=100), P(KnetArray{Float32,2}(125,30000)), P(KnetArray{Float32,2}(2,100)))

In [10]:
# 29s
@info("Training...")
@time for epoch in 1:EPOCHS
    @time adam!((x,y)->loss(model,x,y), dtrn, lr=LR, beta1=BETA_1, beta2=BETA_2, eps=EPS)
end

┌ Info: Training...
└ @ Main In[10]:2


  4.760009 seconds (686.56 k allocations: 90.851 MiB, 0.85% gc time)
  4.225123 seconds (683.99 k allocations: 90.816 MiB, 0.64% gc time)
  4.236897 seconds (684.67 k allocations: 90.788 MiB, 0.85% gc time)
 13.233227 seconds (2.07 M allocations: 272.768 MiB, 0.83% gc time)


In [11]:
@info("Testing...")
@time accuracy(x->predict(model,x), data=minibatch(xtst,ytst,BATCHSIZE))

┌ Info: Testing...
└ @ Main In[11]:1


  2.739723 seconds (3.90 M allocations: 250.743 MiB, 2.82% gc time)


0.854326923076923